In [ ]:
#!pip install textblob
#!pip install pycountry #this is to standardize all country names
#!pip install pandas openpyxl requests beautifulsoup4 spacy transformers pyvis

In [1]:
import os
import PyPDF2
import pandas as pd
import networkx as nx
from collections import defaultdict
from pathlib import Path
import spacy
import nltk
import tensorflow as tf
from transformers import pipeline
from textblob import TextBlob
from pyvis.network import Network
import pycountry
net = Network(notebook=True)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [2]:
#!python -m spacy download en_core_web_sm

In [5]:
# Load the uploaded CSV file
file_path = "/Users/benitaleonardi/Documents/news_excerpts_parsed.csv"
df = pd.read_csv(file_path)

# Display the first few rows to understand the structure
df.head()

,Link,Text
0,https://edition.cnn.com/2023/09/29/business/st...,Starbucks violated federal labor law when it i...
1,https://www.channelnewsasia.com/singapore/su-w...,The first suspect to plead guilty in Singapore...
2,https://edition.cnn.com/2023/05/22/tech/meta-f...,Meta has been fined a record-breaking €1.2 bil...
3,https://www.channelnewsasia.com/singapore/bill...,SINGAPORE: A 45-year-old man linked to Singapo...
4,https://edition.cnn.com/2024/03/05/politics/li...,The Department of Education imposed a record $...


In [9]:
import requests
from bs4 import BeautifulSoup
import spacy
import pycountry
from collections import defaultdict
from transformers import pipeline
from pyvis.network import Network
import pandas as pd

# Load SpaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Load Hugging Face Transformers Sentiment Model
sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

class CountrySentimentAnalyzer:
    def __init__(self):
        """Initialize the text analyzer with NLP and data structures."""
        self.nlp = nlp  # Use loaded SpaCy model
        self.entities = defaultdict(set)
        self.relationships = []

    def standardize_country_name(self, country):
        """Attempt to standardize country names using pycountry."""
        try:
            return pycountry.countries.lookup(country).name
        except LookupError:
            return None  # Ignore non-country entities

    def get_sentiment(self, sentence):
        """Analyze sentiment of a sentence using Transformers."""
        result = sentiment_pipeline(sentence)
        sentiment_label = result[0]['label']  # 'POSITIVE' or 'NEGATIVE'
        confidence = result[0]['score']

        # If confidence is low, classify as NEUTRAL
        if confidence < 0.7:
            return 'NEUTRAL'
        return sentiment_label

    def extract_entities_and_relationships(self, text, source=None):
        """Extract country entities (GPE) and relationships from text."""
        doc = self.nlp(text)
        country_entities = set()

        for ent in doc.ents:
            if ent.label_ == "GPE":
                country_name = self.standardize_country_name(ent.text)
                if country_name:  # Only add valid countries
                    country_entities.add((country_name, source))

        # Store valid countries
        for country in country_entities:
            self.entities['GPE'].add(country)

        # Create relationships between countries in the same sentence
        for sent in doc.sents:
            sent_doc = self.nlp(sent.text)
            entities_in_sent = [self.standardize_country_name(e.text) for e in sent_doc.ents if e.label_ == "GPE"]
            entities_in_sent = [e for e in entities_in_sent if e]  # Remove None values

            if len(entities_in_sent) >= 2:
                sentiment = self.get_sentiment(sent.text)  # Get sentiment of the sentence
                for i in range(len(entities_in_sent) - 1):
                    self.relationships.append({
                        'source': entities_in_sent[i],
                        'target': entities_in_sent[i + 1],
                        'sentence': sent.text,
                        'sentiment': sentiment
                    })

    def scrape_webpage(self, url):
        """Scrape webpage content from a given URL."""
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")
            paragraphs = [p.text for p in soup.find_all("p")]
            return " ".join(paragraphs)  # Return all text as a single string
        except requests.RequestException as e:
            print(f"Failed to fetch {url}: {e}")
            return ""

    def process_csv(self, csv_file):
        """Process CSV file and analyze extracted text."""
        df = pd.read_csv(csv_file)

        for _, row in df.iterrows():
            link = row.get("Link", "").strip()
            text = row.get("Text", "").strip()

            if link:
                print(f"Scraping: {link}")
                web_text = self.scrape_webpage(link)
                self.extract_entities_and_relationships(web_text, source=link)
            
            if text:
                self.extract_entities_and_relationships(text, source="CSV Data")

    def visualize_relationships(self, output_file="country_network.html"):
        """Visualize country relationships using Pyvis with sentiment-based colors."""
        net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, cdn_resources='in_line')

        # Add nodes (countries)
        countries = set()
        for relation in self.relationships:
            countries.add(relation['source'])
            countries.add(relation['target'])

        for country in countries:
            net.add_node(country, label=country, color="blue", size=15)

        # Add edges with sentiment-based color
        for relation in self.relationships:
            sentiment = relation['sentiment']
            if sentiment == "POSITIVE":
                edge_color = "green"
            elif sentiment == "NEGATIVE":
                edge_color = "red"
            else:
                edge_color = "gray"

            net.add_edge(relation['source'], relation['target'], width=2, color=edge_color, title=relation['sentence'])

        # Save and show visualization
        net.show(output_file)

if __name__ == "__main__":
    csv_file = "/Users/benitaleonardi/Documents/news_excerpts_parsed_mini.csv"

    analyzer = CountrySentimentAnalyzer()

    # Process CSV data (scrape web content and analyze text)
    analyzer.process_csv(csv_file)

    # Visualize the combined network from CSV data
    analyzer.visualize_relationships("country_network_2.html")


Device set to use mps:0


Scraping: https://edition.cnn.com/2023/09/29/business/starbucks-union-wages/index.html
Scraping: https://www.channelnewsasia.com/singapore/su-wenqiang-pleads-guilty-billion-dollar-money-laundering-convicted-4234731
Scraping: https://edition.cnn.com/2023/05/22/tech/meta-facebook-data-privacy-eu-fine/index.html
Scraping: https://www.channelnewsasia.com/singapore/billion-dollar-money-laundering-case-zhang-ruijin-sentenced-15-months-jail-4302416
Scraping: https://edition.cnn.com/2024/03/05/politics/liberty-university-fined-campus-safety/index.html
Scraping: https://www.euronews.com/2024/02/23/judge-convicts-former-austrian-chancellor-sebastian-kurz
Scraping: https://edition.cnn.com/2022/07/21/economy/china-fines-didi-data-law-violation-intl-hnk/index.html
Scraping: https://www.brusselstimes.com/justice-belgium/1011990/two-alleged-leaders-of-vast-drug-trafficking-operation-looking-at-20-years-in-prison
Scraping: https://www.expats.cz/czech-news/article/former-czech-mp-gets-three-year-jail-t

In [ ]:

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Load Hugging Face Transformers Sentiment Model
sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

class PDFTextAnalyzer:
    def __init__(self):
        """Initialize the text analyzer with NLP and data structures."""
        self.nlp = nlp  # Use loaded SpaCy model
        self.entities = defaultdict(set)
        self.relationships = []

    def standardize_country_name(self, country):
        """Attempt to standardize country names using pycountry."""
        try:
            return pycountry.countries.lookup(country).name
        except LookupError:
            return None  # Ignore non-country entities

    def get_sentiment(self, sentence):
        """Analyze sentiment of a sentence using Transformers."""
        result = sentiment_pipeline(sentence)
        sentiment_label = result[0]['label']  # 'POSITIVE' or 'NEGATIVE'
        confidence = result[0]['score']

        # If confidence is low, classify as NEUTRAL
        if confidence < 0.7:
            return 'NEUTRAL'
        return sentiment_label

    def extract_entities_and_relationships(self, text, source=None):
        """Extract country entities (GPE) and relationships from text."""
        doc = self.nlp(text)
        country_entities = set()

        for ent in doc.ents:
            if ent.label_ == "GPE":
                country_name = self.standardize_country_name(ent.text)
                if country_name:  # Only add valid countries
                    country_entities.add((country_name, source))

        # Store valid countries
        for country in country_entities:
            self.entities['GPE'].add(country)

        # Create relationships between countries in the same sentence
        for sent in doc.sents:
            sent_doc = self.nlp(sent.text)
            entities_in_sent = [self.standardize_country_name(e.text) for e in sent_doc.ents if e.label_ == "GPE"]
            entities_in_sent = [e for e in entities_in_sent if e]  # Remove None values

            if len(entities_in_sent) >= 2:
                sentiment = self.get_sentiment(sent.text)  # Get sentiment of the sentence
                for i in range(len(entities_in_sent) - 1):
                    self.relationships.append({
                        'source': entities_in_sent[i],
                        'target': entities_in_sent[i + 1],
                        'sentence': sent.text,
                        'sentiment': sentiment
                    })

    def extract_text_from_pdf(self, pdf_file):
        """Extract text from a PDF file using PyPDF2."""
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            extracted_text = page.extract_text()
            if extracted_text:
                text += extracted_text
        return text

    def process_directory(self, directory):
        """Process all PDFs in a directory."""
        for filename in os.listdir(directory):
            if filename.endswith(".pdf"):
                pdf_path = os.path.join(directory, filename)
                with open(pdf_path, "rb") as pdf_file:
                    text = self.extract_text_from_pdf(pdf_file)
                    self.extract_entities_and_relationships(text, source=filename)

    def visualize_relationships(self, output_file="country_network.html"):
        """Visualize country relationships using Pyvis with sentiment-based colors."""
        net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, cdn_resources='in_line')

        # Add nodes (countries)
        countries = set()
        for relation in self.relationships:
            countries.add(relation['source'])
            countries.add(relation['target'])

        for country in countries:
            net.add_node(country, label=country, color="blue", size=15)

        # Add edges with sentiment-based color
        for relation in self.relationships:
            sentiment = relation['sentiment']
            if sentiment == "POSITIVE":
                edge_color = "green"
            elif sentiment == "NEGATIVE":
                edge_color = "red"
            else:
                edge_color = "gray"

            net.add_edge(relation['source'], relation['target'], width=2, color=edge_color, title=relation['sentence'])

        # Save and show visualization
        net.show(output_file)

if __name__ == "__main__":
    pdf_directory = "/Users/benitaleonardi/Downloads/Datathon pdfs"
    analyzer = PDFTextAnalyzer()
    analyzer.process_directory(pdf_directory)  # Process all PDFs in the specified directory
    analyzer.visualize_relationships("country_network.html")  # Visualize the relationships between countries
